In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [2]:
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#config = tf.compat.v1.ConfigProto(device_count = {'GPU': 1})

# To Do

## Preliminary work on the dataset

In [3]:
yahoo_df = pd.read_csv("../data/GOLD.csv")
yahoo_df = yahoo_df.set_index("Date")
yahoo_df = yahoo_df.astype(float,errors="raise")
yahoo_df

,Open,High,Low,Close,vwap,SO,ROC_1,ROC_2,ROC_3,ROC_4,...,SRSI_10,SRSI_14,SRSI_30,Williams_14,ATR_14,CCI,APO,Differenced,Previous_differenced,principalDf
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-10,264.700012,264.700012,264.700012,264.700012,271.220609,0.000000,-1.046724,-1.231339,-1.231339,-0.972681,...,0.097033,0.388087,0.193531,-100.000000,1.340838,-135.011538,-0.635898,-0.010467,-0.001866,5.891493
2001-01-11,264.000000,264.000000,264.000000,264.000000,273.559102,0.000000,-0.264455,-1.308411,-1.492537,-1.492537,...,-0.005937,0.192043,0.177749,-100.000000,1.255395,-129.753670,-1.186539,-0.002645,-0.010467,6.202687
2001-01-12,263.899994,263.899994,263.899994,263.899994,273.600799,0.000000,-0.037881,-0.302236,-1.345797,-1.529853,...,0.034205,0.094658,0.317763,-100.000000,1.101343,-116.122111,-1.786540,-0.000379,-0.002645,5.705446
2001-01-16,263.299988,263.299988,263.299988,263.299988,272.958834,0.000000,-0.227361,-0.265156,-0.528910,-1.570098,...,-0.008394,0.084610,0.076589,-100.000000,1.034498,-113.424908,-2.355132,-0.002274,-0.000379,6.149956
2001-01-17,263.200012,263.200012,263.200012,263.200012,270.550003,0.000000,-0.037970,-0.265245,-0.303026,-0.566679,...,0.047434,0.049092,0.135008,-100.000000,0.909895,-102.043937,-2.678850,-0.000380,-0.002274,5.821742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24,1963.599976,1964.000000,1961.599976,1961.599976,1997.968408,40.575311,1.290922,2.129433,1.711086,1.732187,...,0.402563,0.488230,0.594940,-66.990286,26.269230,24.477209,10.952558,0.012909,0.008278,-2.894129
2022-03-25,1953.800049,1953.800049,1953.800049,1953.800049,1997.689272,34.670747,-0.397631,0.888158,1.723335,1.306651,...,0.419588,0.425315,0.583146,-71.723250,23.806656,3.277338,5.062171,-0.003976,0.012909,-1.428468
2022-03-28,1932.599976,1939.599976,1932.599976,1939.599976,1999.482419,23.921258,-0.726793,-1.121533,0.154911,0.984018,...,0.441452,0.302190,0.559679,-79.204099,23.459112,-33.902231,-1.308983,-0.007268,-0.003976,1.188695


# Train, test, val

In [4]:
X = yahoo_df[['ROC_3', 'Momentum', 'RSI_2', 'ATR_14']]
y = yahoo_df["Up down"]

In [5]:
# Split in 80/20 the dataframeX
X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, test_size = 0.20,
                                                          shuffle=False)

# Split in 75/25 the remaining 80 %
X_train, X_valid, y_train, y_valid = train_test_split(X_train_80, y_train_80,
                                                      test_size = 0.25, shuffle=False)

# Normalize

In [6]:
def normalize(df):
    #df = data
    for column in df:
        df[column]=((df[column]-df[column].mean())/df[column].std())
    return df

In [7]:
X_train_80 = normalize(X_train_80)
X_test = normalize(X_test)


In [8]:
X_train = normalize(X_train)
X_valid = normalize(X_valid)

# Light Gradient Boosting Machine


In [9]:
# Build LightGBM Model
valid_data = lgb.Dataset(X_valid, label = y_valid)
train_data = lgb.Dataset(X_train, label = y_train)
test_data = lgb.Dataset(X_train_80, label = y_train_80)

In [10]:
# Select Hyper-Parameters
params = {'boosting_type': 'dart',
          "num_leaves":[2],
          'max_depth' : [10],
          'objective': 'multiclass',
          'nthread': 4,
          'learning_rate': 0.0125,
          'max_bin': 100,
          'reg_lambda': 1.4,
          'num_class' : 2,  
          'metric' : 'multi_logloss'}

In [11]:
# Train the model
lgbm = lgb.train(params, train_data)

# Predict
pred_train = lgbm.predict(X_train)
pred_valid = lgbm.predict(X_valid)

y_pred_train = np.argmax(pred_train, axis = 1)
y_pred_valid = np.argmax(pred_valid, axis = 1)

# Compute accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000451 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 400
[LightGBM] [Info] Number of data points in the train set: 3192, number of used features: 4
[LightGBM] [Info] Start training from score -0.497444
[LightGBM] [Info] Start training from score -0.936704
Train Accuracy: 0.60808 - Validation Accuracy: 0.53477


# Tuning very slow

In [12]:

param_test ={
            'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 100

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

gs.fit(X_train, y_train)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

In [13]:
params = {'boosting_type': 'dart',
          "learning_rate":0.001,
          "num_iterations":1000,
        'colsample_bytree': 0.9242593373375023,
          'min_child_samples': 282,
          'min_child_weight': 10.0, 
          'num_leaves': 36, 
          'reg_alpha': 0, 
          'reg_lambda': 0.1, 
          'subsample': 0.4955174534902478,
          'objective': 'multiclass',
          'metric' : 'multi_logloss',
          'num_class' : 2, #Away, Draw and Home,
         }

In [14]:
len(pred_train)

3192

In [15]:
# Train the model
lgbm = lgb.train(params, train_data)

# Predict
pred_train = lgbm.predict(X_train)
pred_valid = lgbm.predict(X_valid)

y_pred_train = np.argmax(pred_train, axis = 1)
y_pred_valid = np.argmax(pred_valid, axis = 1)

# Compute accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000123 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 400
[LightGBM] [Info] Number of data points in the train set: 3192, number of used features: 4
[LightGBM] [Info] Start training from score -0.497444
[LightGBM] [Info] Start training from score -0.936704
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Li

C:\Users\enric\anaconda3\envs\data_science\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [16]:
# Predict
pred_test = lgbm.predict(X_test)

y_pred_test = np.argmax(pred_test, axis = 1)

# Compute accuracy
test_acc = accuracy_score(y_test, y_pred_test)
print("Test Accuracy: {:.5f}".format(test_acc))

Test Accuracy: 0.53427


# Save accuracies

In [17]:
# Train the test set

# Predict
pred = lgbm.predict(X_test)
y_pred = np.argmax(pred, axis = 1)

# Compute accuracy
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.5f}".format(acc))

# Store the test and the validation accuracy
LGBM = acc

Test Accuracy: 0.53427


In [18]:
df_prediction = pd.read_csv("../data/model_accuracy_gold.csv", index_col = False)
df_prediction =  df_prediction.drop(columns=["Unnamed: 0"])

In [19]:
#df_prediction = pd.read_csv("../data/model_accuracy.csv", index_col = False)
df_prediction["LGBM_pred"] = y_pred
df_prediction["LGBM_accuracy"] = [LGBM for x in range(len(y_pred))]
df_prediction["LGBM_prob"] = pred[:,1]
df_prediction

,LogReg_pred,LogReg_accuracy,LogReg_prob,RNN_pred,RNN_accuracy,RNN_prob,LSTM_price_accuracy,RF_pred,RF_accuracy,RF_prob,SVC_pred,SVC_accuracy,SVC_prob,LGBM_pred,LGBM_accuracy,LGBM_prob
0,0,0.534272,0.435015,0,0.539906,0.440501,0.829887,0,0.537089,0.348535,0,0.534272,0.402208,0,0.534272,0.417784
1,0,0.534272,0.438317,0,0.539906,0.438469,0.829887,0,0.537089,0.321573,0,0.534272,0.425238,0,0.534272,0.417784
2,0,0.534272,0.441080,0,0.539906,0.438368,0.829887,1,0.537089,0.586142,0,0.534272,0.411844,0,0.534272,0.417784
3,0,0.534272,0.435446,0,0.539906,0.437083,0.829887,0,0.537089,0.413046,0,0.534272,0.406224,0,0.534272,0.417784
4,0,0.534272,0.430490,0,0.539906,0.403022,0.829887,0,0.537089,0.437003,0,0.534272,0.391746,0,0.534272,0.431615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,1,0.534272,0.528832,0,0.539906,0.486045,0.829887,0,0.537089,0.337631,0,0.534272,0.384227,0,0.534272,0.463188
1061,1,0.534272,0.523879,1,0.539906,0.510491,0.829887,0,0.537089,0.305151,0,0.534272,0.392646,0,0.534272,0.478714
1062,1,0.534272,0.521895,0,0.539906,0.478003,0.829887,0,0.537089,0.498127,0,0.534272,0.408522,0,0.534272,0.489069
1063,1,0.534272,0.517769,0,0.539906,0.462106,0.829887,1,0.537089,0.564314,0,0.534272,0.426515,0,0.534272,0.484030


In [20]:
df_prediction.to_csv("../data/model_accuracy_gold.csv")
